# Global Climate Fund Project Portfolio Scraper

Jose Dia

From: https://www.greenclimate.fund/projects

In [123]:
# Packages
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
from cpi_tools import aws_tools

# S3 bucket
# S3_NZFT_BUCKET = 'cpi-nzft'
# year = 2023

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

False

### Project html

In [170]:
# url = 'https://www.greenclimate.fund/project/fp231'
url = 'https://www.greenclimate.fund/project/sap037'

driver = webdriver.Chrome()
driver.get(url)


Define a function for each piece of data

In [166]:
def get_project_name(project):
    return project.find_element(By.XPATH, '/html/body/main/div[2]/div/div/article/div[1]/header/div/p').text

def get_project_description(project):
    return project.find_element(By.XPATH, '//*[@id="overview"]/div[2]/div/section').text

def get_status(project):
    return project.find_element(By.XPATH, '//*[@id="overview"]/div[1]/div/div[2]/ul/li[1]/span[2]').text

def get_approval_date(project):
    return project.find_element(By.XPATH, '//*[@id="overview"]/div[1]/div/div[2]/ul/li[2]/a/span/span').text

def get_est_completion_date(project):
    try:
        return project.find_element(By.XPATH, '//*[@id="overview"]/div[1]/div/div[2]/ul/li[3]/span[2]/span').text
    except:
        return ''
#use_cpi
def get_theme(project):
    return project.find_element(By.XPATH, '//*[@id="impact"]/div/div[2]/div[1]/div/h3').text

#value_cpi
def get_gcf_value_instrument(project): 
    inv_values = project.find_element(By.XPATH, '//*[@id="details"]/div[3]/div[2]/table[1]/tbody').find_elements(By.CSS_SELECTOR, 'tr')
    
    val_instrument = []
    for sub_element in inv_values:
        val_instrument.append(sub_element.text)  
    return val_instrument 

def get_co_financing(project):
    inv_values = project.find_element(By.XPATH, '//*[@id="details"]/div[3]/div[2]/table[3]/tbody').find_elements(By.CSS_SELECTOR, 'tr')
    
    co_financing = []
    for sub_element in inv_values:
        co_financing.append(sub_element.text)
    return co_financing

    # sector_cpi
def get_result_areas(project): 
    result_areas = project.find_element(By.XPATH, '//*[@id="impact"]/div/div[2]/div[2]/div/ul').find_elements(By.CSS_SELECTOR, 'li')
    areas = []
    for sub_element in result_areas:
        if sub_element.find_element(By.XPATH, './/a/div').get_attribute('class')=='icons active':
            areas.append(sub_element.find_element(By.XPATH, './/a').get_attribute('data-original-title'))
        else:
            areas.append('')
    return areas

#region_original
def get_region(project):
    project.find_element(By.XPATH, '/html/body/main/div[2]/div/div/article/div[3]/div[3]/div/div[1]/div[2]/ul[1]/li').text
    
#country_original
def get_country(project):
    project.find_element(By.XPATH, '//*[@id="details"]/div[2]/ul[2]/li/a').text

In [163]:
details = driver.find_element(By.XPATH, '//*[@id="details"]/div[2]/ul[2]/li/a').text#.find_elements(By.CSS_SELECTOR, 'tr')
    
details

'Ghana'

In [171]:
row_data = {}
project = driver

row_data['project_name'] = get_project_name(project)

row_data['project_description'] = get_project_description(project)

row_data['status'] = get_status(project)

row_data['approval_date'] = get_approval_date(project)

row_data['est_completion_date'] = get_est_completion_date(project)

row_data['theme'] = get_theme(project)

row_data['gcf_valUSD_instrument'] = get_gcf_value_instrument(project) 

row_data['co_finance_valUSD_instrument'] = get_co_financing(project)

row_data['result_areas'] = get_result_areas(project)

row_data['region'] = get_region(project)

row_data['country'] = get_country(project)

In [172]:
row_data

{'project_name': 'Avaana Sustainability Fund',
 'project_description': "India, with one-sixth of the world's population, sees 17 out of every 20 people vulnerable to climate risks, with five living in areas extremely vulnerable to climate change. The country's dependence on rain-fed agriculture, long coastline, and low social safety nets contribute to its high vulnerability. India's carbon dioxide emissions have been increasing, with a projected doubling by 2030 in line with economic growth. To boost its economy while reducing emissions and increasing resilience, India urgently requires a platform dedicated to climate technology. This platform should frequently test innovative climate technologies and business models, scaling them to demonstrate viable and affordable solutions that serve the most vulnerable communities. However, the ecosystem for climate technology ventures in India is still nascent.  \nThe Avaana Sustainability Fund (ASF) is a USD 120 million venture capital fund that

In [173]:
driver.quit()

In [87]:
# # Find the 'List' button to get all investors
# list_button = WebDriverWait(driver, 10).until(
#         EC.element_to_be_clickable((By.XPATH, '//li[@ng-repeat="tab in rightPanelTabs"][3]'))
#     )
# list_button.click()
# time.sleep(3)

# # Find all list items
# list_items = driver.find_elements(By.XPATH, '//ul[@class="list-unstyled ng-scope"]/li[starts-with(@id, "item-")]')
# time.sleep(2)

# data = {}
# list_len = len(list_items) 

# # Click on each list item and scrape the data, save in data
# for index, item in enumerate(list_items):
#     row_data = {}

#     # Get investor name and click on this investor
#     investor_name = item.find_element(By.TAG_NAME, 'h6').text
#     item.click()

#     # Get investor details
#     details = driver.find_element(By.XPATH, '//div[@ng-if="nodeRightInfo"]')

#     # Extract description
#     row_data['description'] = extract_description(details)

#     # Extract keywords
#     row_data['keywords'] = extract_keywords(details)

#     # Extract tags
#     extract_tags(details, row_data)

#     # Extract other info
#     extract_info(details, row_data)
    
#     # Extract Co-investors
#     row_data['co-investors'] = extract_coinvestors(details)

#     # Add the row_data to the main dataset
#     data[investor_name] = row_data

#     # Print out the progress every 10%
#     # Calculate the current progress percentage
#     progress_percentage = (index + 1) / list_len * 100
#     if progress_percentage % 10 == 0:
#         print(f"Progress: {progress_percentage:.0f}%")

#     # Add a delay
#     time.sleep(1)


# driver.quit()
    

Progress: 50%
Progress: 100%


In [90]:
# df = pd.DataFrame.from_dict(data, orient='index')
# df

,description,keywords,Top Funding Type,Investor Country,Investor City,Co-Investor Cluster Theme,Co-Investor Cluster Hubs,Avg Round Size,Median Round Size,# Co-Investors,# Portfolio Companies,Portfolio Manufacturing Mentions,co-investors,Portfolio Equity-Justice Mentions
#Angels,<h1>ANGELS is an investment collective. In add...,"[resilience, data, transportation, mobility, p...",Series A,United States,San Francisco,"data, artificial intelligence, mobility, vehic...","SVA, Greycroft, Evolution VC Partners","17,800,000","17,800,000",57,3,0.333,"[10x Group, Accel, American Family Insurance I...",NaN
(ERA) Entrepreneurs Roundtable Accelerator,(ERA) Entrepreneurs Roundtable Accelerator is ...,"[data, carbon, emissions, food, energy, farmin...",Seed,United States,New York,"manufacturing, water, energy storage, renewabl...","National Science Foundation, Techstars, MassCh...","118,182","100,000",12,11,NaN,"[Artesian VC, Brinc, Everywhere Ventures (The ...",0.09
10X Capital,"10X invests across the capital structure, alon...","[electric, food, mobility, satellite, energy, ...",Series A,United States,New York,"data, carbon, emissions, artificial intelligen...","My Climate Journey Collective, Bossanova Inves...","18,583,333","20,500,000",91,10,0.4,"[ACME Capital, ACRE, Accelr8, Alliance of Ange...",0.2
10x Group,10x Group is a group of serial entrepreneurs a...,"[solar, data, energy, plastic, renewable energ...",Seed,Germany,Munich,"data, carbon, emissions, energy, manufacturing","Y Combinator, Soma Capital, Liquid 2 Ventures",NaN,NaN,23,2,0.5,"[Ajax Strategies, Brainchild Holdings, CSC Ups...",NaN
14W,"Founded in 2010, 14W is a global venture capit...","[artificial intelligence, shipping, data, circ...",Series C,United States,New York,"data, artificial intelligence, mobility, vehic...","SVA, Greycroft, Evolution VC Partners","21,093,590","17,187,180",21,2,NaN,"[3L Capital, BlackRock, Broadway Angels, Canaa...",0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mHUB,mHUB is a leading independent hardtech and man...,"[energy, industrial, carbon, solar, energy sto...",Pre-Seed,United States,Chicago,"manufacturing, water, energy storage, renewabl...","National Science Foundation, Techstars, MassCh...","75,000","75,000",8,7,0.143,"[CanopyBoulder, Cleantech Open, LongJump, Righ...",0.24
v1.vc,V1.VC offers in investing in early-stage inter...,"[accountability, data, forecasting, circular e...",Seed,United States,San Francisco,"data, carbon, emissions, artificial intelligen...","My Climate Journey Collective, Bossanova Inves...","2,000,000","2,000,000",22,2,NaN,"[Blackhorn Ventures, Climate Capital, GRIL Ven...",NaN
venVelo,venVelo directs the successful entrepreneur th...,"[mobility, cleantech, transportation, cities, ...",Seed,United States,Content not found,"coastal, technology incubator, distillation te...","Omidyar Network, Daniel Scrivner, Rashaun Will...","2,850,000","2,850,000",5,2,NaN,"[Florida Opportunity Fund, For Good Ventures, ...",NaN
zipdragon ventures,Zipdragon ventures (ZDV) is an investment firm...,"[energy storage, data, battery, buildings, ene...",Venture (Unknown Stage),United States,San Francisco,"manufacturing, water, energy storage, renewabl...","National Science Foundation, Techstars, MassCh...",NaN,NaN,11,2,0.5,"[ACRE, Anzu Partners, Bee Partners, Darling Ve...",NaN


### Write to S3

In [92]:
# # Processed data path in S3 Bucket
# processed_data_s3_path = f'{year}/raw/nzft_indicators'

# # Processed filename in S3
# processed_data_s3_filename = 'Climate_Co-investor_Tracker'

# aws_tools.write_to_s3(df, S3_NZFT_BUCKET, processed_data_s3_path, processed_data_s3_filename)